# Create random sample (to give to Michael)
Here's how we created the random sample (i.e. various halos in LJ, some are FGs, most are not). It is built to include three narrow mass bins (same as the default bins in `make_masks()` and in each mass bin, there should be as many halos (random) as there are total number of fossils in that halo (as calculated from the `fg_forest`)

## Set Up

In [2]:
import haccytrees.mergertrees
import h5py
import numpy as np
import numba
import scipy
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
from matplotlib.lines import Line2D
import matplotlib.colors as mcolors
import matplotlib.patches as mpatches
import pandas as pd
from astropy.cosmology import FlatLambdaCDM
import astropy.units as u
from itertools import groupby
from matplotlib.ticker import ScalarFormatter
%load_ext line_profiler
%reload_ext autoreload
%autoreload 1
%aimport help_func_haccytrees

plt.rcParams.update({
    "text.usetex": True,
    'font.size': 13,
    "figure.figsize": (5.25, 3.5),#(6.25, 4.25), #(6.25, 3.75)
    "patch.linewidth": 1
})

pyfftw not available, using numpy fft


#### What is `data`?
`data` is a dictionary containing only the halo information we are interested in (i.e. just at $z=0$). Use `data.keys()` to see what's inside. You can treat `data` largely like you treated `forest`: i.e. to get a `key` for a specific subset of halos, use `data['key_name'][halo_idx]`.
On the fossil groups side, the equivalent of `data` is `fg_catalog`, which consists only of $z=0$ information for halos we have previously identified as fossil groups. We create `fg_catalog` from `fg_forest.hdf5` below.

In [3]:
%%time
with h5py.File("/data/a/cpac/mbuehlmann/LastJourney/forest/z0_catalog.hdf5", "r") as f:
    data = {
        k: d[:] for k, d in f.items()
    }
s = np.argsort(data['tree_node_index'])
data = {k: data[k][s] for k in data.keys()} # all halos at z=0 above a certain mass threshold

CPU times: user 31.1 s, sys: 19.7 s, total: 50.8 s
Wall time: 50.7 s


In [6]:
data.keys()

dict_keys(['delta+1_rsoft0.0', 'delta+1_rsoft1.0', 'delta+1_rsoft10.0', 'delta+1_rsoft2.0', 'delta+1_rsoft4.0', 'filenum', 'index', 'sig_rsoft_0.0', 'sig_rsoft_1.0', 'sig_rsoft_10.0', 'sig_rsoft_2.0', 'sig_rsoft_4.0', 'sod_halo_cdelta', 'sod_halo_cdelta_accum', 'sod_halo_cdelta_peak', 'tree_node_index', 'tree_node_mass', 'x', 'xoff_com', 'xoff_fof', 'xoff_sod', 'y', 'z'])

### Now create `fg_catalog` (similar to `data`)

In [5]:
%%time
fg_forest_lm5, fg_progenitor_array_lm5 = haccytrees.mergertrees.read_forest(
    "/data/a/cpac/mbuehlmann/LastJourney/forest/fg_forest.hdf5",
    'LastJourney',
    mass_threshold=5e11
)

CPU times: user 25.4 s, sys: 33.5 s, total: 58.8 s
Wall time: 58.8 s


In [7]:
%%time
mask_z0 = fg_forest_lm5['snapnum'] == 100
fg_catalog_lm5 = {k: fg_forest_lm5[k][mask_z0] for k in fg_forest_lm5.keys()} # forest data at z=0 # Not sure why this different from `assign_fgs()`?
s = np.argsort(fg_catalog_lm5['tree_node_index']) # How is this being sorted exactly?
fg_catalog_lm5 = {k: fg_catalog_lm5[k][s] for k in fg_catalog_lm5.keys()}

CPU times: user 2.36 s, sys: 112 ms, total: 2.47 s
Wall time: 2.47 s


In [9]:
print(fg_catalog_lm5.keys())

dict_keys(['branch_size', 'desc_node_index', 'fof_halo_center_x', 'fof_halo_center_y', 'fof_halo_center_z', 'fof_halo_count', 'fof_halo_mass', 'fof_halo_tag', 'snapnum', 'sod_halo_cdelta', 'sod_halo_cdelta_accum', 'sod_halo_cdelta_error', 'sod_halo_cdelta_peak', 'sod_halo_count', 'sod_halo_mass', 'sod_halo_radius', 'tree_node_index', 'tree_node_mass', 'xoff_com', 'xoff_fof', 'xoff_sod', 'scale_factor', 'descendant_idx', 'progenitor_count', 'progenitor_offset', 'halo_index'])


In [10]:
# To create an index of fossil groups
@numba.jit(nopython=True)
def assign_fgs(full_tn_index, fg_tn_index):
    n_full = len(full_tn_index)
    n_fg = len(fg_tn_index)
    c_full_idx = 0
    c_fg_idx = 0
    fg_index = np.empty(n_fg, dtype=np.int64)
    fg_index[:] = -1
    while c_full_idx < n_full and c_fg_idx < n_fg:
        if full_tn_index[c_full_idx] == fg_tn_index[c_fg_idx]:
            fg_index[c_fg_idx] = c_full_idx
            c_fg_idx += 1
        else:
            c_full_idx += 1
    return fg_index
fg_idx = assign_fgs(data['tree_node_index'], fg_catalog_lm5['tree_node_index'])

### Create random sample

In [13]:
tn_idx = [] # tree node index
mass = []
idx = []
file = []
for this_bin in [[1e13, 10**13.05], [10**13.3, 10**13.35], [10**13.6, 10**13.65]]:
    bin_mask = (data['tree_node_mass'] > this_bin[0]) & (data['tree_node_mass'] < this_bin[1])
    bin_mask_idx, = np.nonzero(bin_mask)
    n_fgs = len(fg_idx[(fg_catalog_lm5['tree_node_mass'] > this_bin[0]) & (fg_catalog_lm5['tree_node_mass'] < this_bin[1])])
    random_sample_mask = np.random.choice(bin_mask_idx, n_fgs, replace = False)
    
    tn_idx.append(data['tree_node_index'][random_sample_mask])
    mass.append(data['tree_node_mass'][random_sample_mask])
    idx.append(data['index'][random_sample_mask])
    file.append(data['filenum'][random_sample_mask])
    
create_file = False
if create_file:
    with h5py.File('random_sample.h5', 'w') as f:
        f.create_dataset('tree_node_index', data=np.concatenate(tn_idx))
        f.create_dataset('tree_node_mass', data=np.concatenate(mass))
        f.create_dataset('index', data=np.concatenate(idx))
        f.create_dataset('file', data=np.concatenate(file))

[array([2247382487065960618, 2247442453399346018, 2247372887814052310, ...,
       2247418367222766344, 2247301406673370823, 2247517276024623655]), array([2247530736452122641, 2247559078941299220, 2247344961936722907, ...,
       2247461042017825579, 2247530178106368471, 2247523727065495203]), array([2247311203493760669, 2247422851168633196, 2247567067580466021, ...,
       2247442002427798725, 2247475060791066866, 2247448238720296175])]
[array([1.0062518e+13, 1.0187519e+13, 1.0630454e+13, ..., 1.0239149e+13,
       1.1005454e+13, 1.1051650e+13], dtype=float32), array([2.0652212e+13, 2.0548950e+13, 2.2380475e+13, ..., 2.0008189e+13,
       2.2029932e+13, 2.1972866e+13], dtype=float32), array([4.2149534e+13, 4.4440299e+13, 4.0089745e+13, ..., 4.0182137e+13,
       4.2462035e+13, 4.2054424e+13], dtype=float32)]
[array([156976971,  39208088,  58070978, ..., 125788208,  57588077,
        78619593]), array([214009278, 118449372, 143200851, ..., 229440714, 222748617,
       143481611]), arra

### Reload random sample

In [3]:
with h5py.File('random_sample.h5', 'r') as hf: # 'r' = read
    print(hf.keys())
    print(len(hf['tree_node_index']))
    bin1 = hf['tree_node_index'][:269358]
    bin2 = hf['tree_node_index'][269358: 269358 + 36181]
    bin3 = hf['tree_node_index'][269358 + 36181: ]
len(bin1) + len(bin2) + len(bin3)

<KeysViewHDF5 ['file', 'index', 'tree_node_index', 'tree_node_mass']>
307993


307993

### Check statistics of the random sample

In [4]:
%%time
forest, progenitor_array = haccytrees.mergertrees.read_forest(
    '/data/a/cpac/mbuehlmann/LastJourney/forest/target_forest_aurora.hdf5',
    'LastJourney', nchunks=1, chunknum=0, #mass_threshold = 2.7*10**11,
    include_fields = ["tree_node_mass", "snapnum", "fof_halo_tag", "sod_halo_cdelta", "fof_halo_center_x", "fof_halo_center_y", "fof_halo_center_z"]
)

CPU times: user 6.77 s, sys: 8.62 s, total: 15.4 s
Wall time: 15.4 s


In [8]:
# Make masks
halo_masks = help_func_haccytrees.make_masks(forest) # Forest should already only contain values in these bins
print(len(halo_masks[0][halo_masks[0]]) + len(halo_masks[1][halo_masks[1]]) + len(halo_masks[2][halo_masks[2]]))

307993


In [12]:
print("actual min: ", 10**13)
print("this min:   ", np.min(forest['tree_node_mass'][np.nonzero(halo_masks[0])]))
print("actual max: ", 10**13.05)
print("this max:   ", np.max(forest['tree_node_mass'][np.nonzero(halo_masks[0])]))

actual min:  10000000000000
this min:    10000018000000.0
actual max:  11220184543019.652
this max:    11220130000000.0


In [14]:
%%time
absolute_threshold = True
threshold = 5e11
z_thresh = 1
norm_tf = False
restrict_mass = True
use_sigma = True
mrich_thresh = 0.1
# Go find yourself some fossil groups!
for i, this_halo_mask in enumerate(halo_masks):
    target_idx = np.nonzero(this_halo_mask)[0]
    mainbranch_index, mainbranch_masses = help_func_haccytrees.get_branches(target_idx, forest)
    mainbranch_mergers = help_func_haccytrees.get_mainbranch_mergers(forest, progenitor_array, mainbranch_index, absolute_threshold)
    major_mergers = help_func_haccytrees.get_major_mergers(mainbranch_mergers, threshold)
    lmm_redshifts = help_func_haccytrees.get_lmms(major_mergers, threshold)
    fgs, qhs, mrich = help_func_haccytrees.find_specials(forest, mainbranch_index, major_mergers, lmm_redshifts, target_idx, z_thresh, mrich_thresh = mrich_thresh, restrict_mass = restrict_mass, use_sigma = use_sigma, mainbranch_masses = mainbranch_masses)
    print("bin/; ", str(i))
    print(len(target_idx), " halos")
    print(len(fgs), " fossils")
    print(len(qhs), " qhs")
    print(len(mrich), " merger rich halos")

0
bin  0
269358  halos
33569  fossils
4171  qhs
15581  merger rich halos
1
bin  1
36181  halos
1364  fossils
9  qhs
2063  merger rich halos
2
bin  2
2454  halos
13  fossils
0  qhs
126  merger rich halos
CPU times: user 3.04 s, sys: 677 ms, total: 3.72 s
Wall time: 2.04 s


In [7]:
print(bin1)
print(len(bin1))
print(bin1[0])

[2247323323891456484 2247413818852391226 2247407427941061735 ...
 2247442053967417332 2247298236987477760 2247393623916173449]
269358
2247323323891456484


In [ ]:
mainbranch_index, mainbranch_masses = help_func_haccytrees.get_branches(bin1[0], forest)